In [25]:
# import time
# time.sleep(5000)

### Import packages

In [23]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset
import optuna
from torch.optim import lr_scheduler
import numpy as np
import pandas as pd
from datetime import datetime
import os
import json

from data_loader import get_cifar10_dataloaders, get_test_dataloader, get_kaggle_test_dataloader
from helper import optimizer_map, scheduler_map, num_params, update_study_details, cifar_10_mean_std
from models import BaseResNet, EfficientNetB0, SmallResNet0, LargeResNet0, SmallResNet1
from trainer import train_model
from run import single_run

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Configure the device

In [24]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [25]:
# Define the objective function for Optuna
def objective(trial):
    study_dir = f"studies/{trial.study.study_name}"
    os.makedirs(study_dir, exist_ok=True) # Create a directory for checkpoints if it doesn't exist

    # Suggest hyperparameters
    num_epochs = trial.suggest_int("num_epochs", 50, 125)
    model_type = trial.suggest_categorical("model_type", ["base","smallresnet", "efficientnet", "largeresnet"])
    batch_size = trial.suggest_categorical("batch_size", [64, 128, 256, 512])
    optimizer_type = trial.suggest_categorical("optimizer_type", ["Adam", "SGD", "AdamW"])
    scheduler_type = trial.suggest_categorical("scheduler_type", ["CosineAnnealingLR", "OneCycleLR", "ReduceLROnPlateau"])
    
    optimizer_params = {}
    if optimizer_type == "SGD":
        optimizer_params["lr"] = trial.suggest_float("lr", 0.001, 0.1, log=True)
        optimizer_params["momentum"] = trial.suggest_float("momentum", 0.8, 0.99)
        optimizer_params["weight_decay"] = trial.suggest_float("weight_decay", 1e-5, 5e-4, log=True)
        scheduler_type = trial.suggest_categorical("scheduler_type", ["CosineAnnealingLR", "OneCycleLR", "ReduceLROnPlateau"])
        optimizer_params["nesterov"] = scheduler_type != "ReduceLROnPlateau"
    
    elif optimizer_type == "Adam":
        optimizer_params["betas"] = (
            trial.suggest_float("beta1", 0.85, 0.95), 
            trial.suggest_float("beta2", 0.99, 0.999)
        )
        optimizer_params["lr"] = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
        optimizer_params["weight_decay"] = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
        if scheduler_type == "OneCycleLR":
            scheduler_type = None

    elif optimizer_type == "AdamW":
        optimizer_params["betas"] = (
            trial.suggest_float("beta1", 0.85, 0.95), 
            trial.suggest_float("beta2", 0.99, 0.999)
        )
        optimizer_params["lr"] = trial.suggest_float("lr", 1e-5, 5e-3, log=True)
        optimizer_params["weight_decay"] = trial.suggest_float("weight_decay", 1e-3, 1e-1, log=True)
        scheduler_type = trial.suggest_categorical("scheduler_type", ["CosineAnnealingLR", "OneCycleLR", "ReduceLROnPlateau"])

    
    # Realistic tranformation for better generalization
    train_transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.05),  # Mild color variations
        transforms.ToTensor(),
        transforms.Normalize(**cifar_10_mean_std),
    ])
        
    train_loader, valid_loader = get_cifar10_dataloaders(
        train_transform,
        subset_percent=1, 
        valid_size=0.1,
        batch_size=batch_size,
        num_workers=8,
        use_kaggle=True
    )

    scheduler_params = {}
    if scheduler_type == "CosineAnnealingLR":
        # num_epochs = trial.suggest_int("num_epochs", 100, 150)
        scheduler_params["T_max"] = num_epochs
        scheduler_params["eta_min"] = trial.suggest_float("eta_min", 1e-6, 1e-3, log=True)
        
    elif scheduler_type == "ReduceLROnPlateau":
        # num_epochs = trial.suggest_int("num_epochs", 75, 125)
        scheduler_params["factor"] = trial.suggest_float("factor", 0.1, 0.5)
        scheduler_params["patience"] = trial.suggest_int("patience", 5, 20)
        scheduler_params["threshold"] = trial.suggest_float("threshold", 0.01, 0.1)
        scheduler_params["mode"] = "min"
        
    elif scheduler_type == "OneCycleLR":
        # num_epochs = trial.suggest_int("num_epochs", 50, 75)
        if optimizer_type == "SGD":
            scheduler_params["max_lr"] = trial.suggest_float("max_lr", 0.01, 0.3)
        else: # AdamW
            scheduler_params["max_lr"] = trial.suggest_float("max_lr", 0.001, 0.01)
        scheduler_params["steps_per_epoch"] = len(train_loader)
        scheduler_params["epochs"] = num_epochs
        scheduler_params["anneal_strategy"] = "cos"

                
    # Select Model
    if model_type == "smallresnet":
        model = SmallResNet0()
    elif model_type == "efficientnet":
        model = EfficientNetB0()
    elif model_type == "largeresnet":
        model = LargeResNet0()
    else:
        model = BaseResNet()
        
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    trial_details = trial.params.copy()
    trial_details["trainable_parameters"] = num_params(model)
    
    # Print trial details
    print("-" * 50)
    print(f"{trial.number=}")
    for param, val in trial_details.items():
        print(f"{param}: {val}")
    print("- " * 25)
    update_study_details(study_dir, trial.number, trial_details)

    optimizer = optimizer_map[optimizer_type](model.parameters(), **optimizer_params)
    scheduler = scheduler_map[scheduler_type](optimizer, **scheduler_params) if scheduler_type else None
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

    # Checkpoint the model with the best validation accuracy
    chkpt_dir = os.path.join(study_dir, "checkpoint")
    plot_dir = os.path.join(study_dir, "plots")
    os.makedirs(chkpt_dir, exist_ok=True)
    os.makedirs(plot_dir, exist_ok=True)

    # Training
    best_val_accuracy = train_model(
        model, train_loader, criterion, optimizer, valid_loader=valid_loader, num_epochs=num_epochs, 
        device=device, scheduler=scheduler, trial=trial, chkpt_dir=chkpt_dir, plot_dir=plot_dir
    )
    
    trial_details["best_val_accuracy"] = best_val_accuracy
    update_study_details(study_dir, trial.number, trial_details)
    return best_val_accuracy

### Start new study

In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
study_name = f"study_{timestamp}"

study = optuna.create_study(
    study_name=study_name,
    storage="sqlite:///study.db",
    direction="maximize",
    load_if_exists=True,
)

study.optimize(objective, n_trials=100)

print("Best trial:", study.best_trial.number)
print("Best hyperparameters:", study.best_params)
print("Best validation accuracy:", study.best_value)

[I 2025-03-12 02:29:53,436] A new study created in RDB with name: study_2025-03-12_02-29-51


--------------------------------------------------
trial.number=0
num_epochs: 113
model_type: smallresnet
batch_size: 512
optimizer_type: AdamW
scheduler_type: CosineAnnealingLR
beta1: 0.9202155036052446
beta2: 0.9977411320853791
lr: 3.494686994613835e-05
weight_decay: 0.01796373082906369
eta_min: 2.4823405413306104e-06
trainable_parameters: 2998402
- - - - - - - - - - - - - - - - - - - - - - - - - 
  Epoch [1/113], Batch [80/88], Train Acc: 16.5234 Loss: 2.1038
  Validation Accuracy after Epoch 1: 24.0200
  Cidar10.1 Accuracy: 22.45
  Epoch [2/113], Batch [80/88], Train Acc: 21.4917 Loss: 2.0582
  Validation Accuracy after Epoch 2: 24.5000
  Cidar10.1 Accuracy: 23.35
  Epoch [3/113], Batch [80/88], Train Acc: 27.2290 Loss: 1.9905
  Validation Accuracy after Epoch 3: 32.2800
  Cidar10.1 Accuracy: 27.3
  Epoch [4/113], Batch [80/88], Train Acc: 32.4854 Loss: 1.9102
  Validation Accuracy after Epoch 4: 35.9600
  Cidar10.1 Accuracy: 29.4
  Epoch [5/113], Batch [80/88], Train Acc: 33.6157 

[I 2025-03-12 02:48:33,594] Trial 0 finished with value: 77.5 and parameters: {'num_epochs': 113, 'model_type': 'smallresnet', 'batch_size': 512, 'optimizer_type': 'AdamW', 'scheduler_type': 'CosineAnnealingLR', 'beta1': 0.9202155036052446, 'beta2': 0.9977411320853791, 'lr': 3.494686994613835e-05, 'weight_decay': 0.01796373082906369, 'eta_min': 2.4823405413306104e-06}. Best is trial 0 with value: 77.5.


--------------------------------------------------
trial.number=1
num_epochs: 99
model_type: efficientnet
batch_size: 128
optimizer_type: Adam
scheduler_type: ReduceLROnPlateau
beta1: 0.879263309837687
beta2: 0.9902483573125087
lr: 1.0489767423709725e-05
weight_decay: 6.32754805913792e-06
factor: 0.11188403904979145
patience: 6
threshold: 0.092594865117007
trainable_parameters: 3599686
- - - - - - - - - - - - - - - - - - - - - - - - - 
  Epoch [1/99], Batch [350/352], Train Acc: 13.0625 Loss: 2.3211
  Validation Accuracy after Epoch 1: 14.4400
  Cidar10.1 Accuracy: 13.95
  Epoch [2/99], Batch [350/352], Train Acc: 14.7612 Loss: 2.2246
  Validation Accuracy after Epoch 2: 15.4200
  Cidar10.1 Accuracy: 15.3
  Epoch [3/99], Batch [350/352], Train Acc: 16.6272 Loss: 2.2201
  Validation Accuracy after Epoch 3: 19.8200
  Cidar10.1 Accuracy: 17.75
  Epoch [4/99], Batch [350/352], Train Acc: 19.0781 Loss: 2.1436
  Validation Accuracy after Epoch 4: 20.7400
  Cidar10.1 Accuracy: 18.95
  Epoch [

[I 2025-03-12 03:16:02,770] Trial 1 finished with value: 37.0 and parameters: {'num_epochs': 99, 'model_type': 'efficientnet', 'batch_size': 128, 'optimizer_type': 'Adam', 'scheduler_type': 'ReduceLROnPlateau', 'beta1': 0.879263309837687, 'beta2': 0.9902483573125087, 'lr': 1.0489767423709725e-05, 'weight_decay': 6.32754805913792e-06, 'factor': 0.11188403904979145, 'patience': 6, 'threshold': 0.092594865117007}. Best is trial 0 with value: 77.5.


...

--------------------------------------------------
trial.number=54
num_epochs: 80
model_type: largeresnet
batch_size: 128
optimizer_type: AdamW
scheduler_type: OneCycleLR
beta1: 0.903403689050069
beta2: 0.9944854353997653
lr: 0.0006508055442592868
weight_decay: 0.00133983275181981
max_lr: 0.004827358361331066
trainable_parameters: 4903242
- - - - - - - - - - - - - - - - - - - - - - - - - 
  Epoch [1/80], Batch [350/352], Train Acc: 45.1116 Loss: 1.5649
  Validation Accuracy after Epoch 1: 53.6600
  Cidar10.1 Accuracy: 44.65
  Epoch [2/80], Batch [350/352], Train Acc: 61.7656 Loss: 1.3281
  Validation Accuracy after Epoch 2: 63.2600
  Cidar10.1 Accuracy: 51.1
  Epoch [3/80], Batch [350/352], Train Acc: 68.6920 Loss: 1.2910
  Validation Accuracy after Epoch 3: 63.5200
  Trial pruned due to no improvement.


[I 2025-03-12 08:14:02,098] Trial 54 pruned. 


--------------------------------------------------
trial.number=55
num_epochs: 75
model_type: smallresnet
batch_size: 128
optimizer_type: AdamW
scheduler_type: OneCycleLR
beta1: 0.8936407149473528
beta2: 0.9966856937168362
lr: 0.0003744134239293919
weight_decay: 0.003928592864932334
max_lr: 0.008498398639597823
trainable_parameters: 2998402
- - - - - - - - - - - - - - - - - - - - - - - - - 
  Epoch [1/75], Batch [350/352], Train Acc: 36.8460 Loss: 1.7281
  Validation Accuracy after Epoch 1: 45.0800
  Trial pruned due to no improvement.


[I 2025-03-12 08:14:15,464] Trial 55 pruned. 


--------------------------------------------------
trial.number=56
num_epochs: 70
model_type: largeresnet
batch_size: 128
optimizer_type: AdamW
scheduler_type: CosineAnnealingLR
beta1: 0.8965602798454313
beta2: 0.9926179784685416
lr: 0.0011127033334149478
weight_decay: 0.0026495466031555013
eta_min: 0.0009446330554198705
trainable_parameters: 4903242
- - - - - - - - - - - - - - - - - - - - - - - - - 
  Epoch [1/70], Batch [350/352], Train Acc: 44.5290 Loss: 1.4570
  Validation Accuracy after Epoch 1: 55.6200
  Cidar10.1 Accuracy: 43.1
  Epoch [2/70], Batch [350/352], Train Acc: 62.9464 Loss: 1.1584
  Validation Accuracy after Epoch 2: 65.4200
  Cidar10.1 Accuracy: 52.7
  Epoch [3/70], Batch [350/352], Train Acc: 70.3750 Loss: 1.2020
  Validation Accuracy after Epoch 3: 72.2000
  Cidar10.1 Accuracy: 59.3
  Epoch [4/70], Batch [350/352], Train Acc: 75.7545 Loss: 1.0654
  Validation Accuracy after Epoch 4: 74.2200
  Cidar10.1 Accuracy: 62.15
  Epoch [5/70], Batch [350/352], Train Acc: 78.

### Resume study
Helps run more studies since we only have 4 hour time limit

In [4]:
# study_2025-03-11_16-30-52 = peaks at 93% without SWA or Lookahead
study_name = "study_2025-03-12_02-29-51"

# Load and continue running trials
study = optuna.create_study(
    study_name=study_name,
    storage="sqlite:///study.db",
    direction="maximize",
    load_if_exists=True
)

study.optimize(objective, n_trials=25)  # Run another batch
print("Continued Study:")
print("Best trial:", study.best_trial.number)
print("Best hyperparameters:", study.best_params)
print("Best validation accuracy:", study.best_value)

[I 2025-03-12 10:04:13,481] Using an existing study with name 'study_2025-03-12_02-29-51' instead of creating a new one.


--------------------------------------------------
trial.number=98
num_epochs: 72
model_type: largeresnet
batch_size: 128
optimizer_type: AdamW
scheduler_type: OneCycleLR
beta1: 0.8797085934596846
beta2: 0.9949971606374951
lr: 0.0004585314299323415
weight_decay: 0.0014826433974480398
max_lr: 0.007995567499122606
trainable_parameters: 4903242
- - - - - - - - - - - - - - - - - - - - - - - - - 
  Epoch [1/72], Batch [350/352], Train Acc: 45.5223 Loss: 1.5296
  Validation Accuracy after Epoch 1: 54.1200
  Cidar10.1 Accuracy: 45.0
  Epoch [2/72], Batch [350/352], Train Acc: 63.0960 Loss: 1.4150
  Validation Accuracy after Epoch 2: 62.1200
  Cidar10.1 Accuracy: 47.45
  Epoch [3/72], Batch [350/352], Train Acc: 70.1987 Loss: 1.0942
  Validation Accuracy after Epoch 3: 68.2400
  Cidar10.1 Accuracy: 52.65
  Epoch [4/72], Batch [350/352], Train Acc: 74.2098 Loss: 1.1175
  Validation Accuracy after Epoch 4: 70.5200
  Cidar10.1 Accuracy: 57.95
  Epoch [5/72], Batch [350/352], Train Acc: 76.7701 Lo

[I 2025-03-12 10:05:05,184] Trial 98 pruned. 


--------------------------------------------------
trial.number=99
num_epochs: 122
model_type: largeresnet
batch_size: 256
optimizer_type: AdamW
scheduler_type: OneCycleLR
beta1: 0.9356739822937291
beta2: 0.9965169612969706
lr: 0.0009038583428644764
weight_decay: 0.002498746736724739
max_lr: 0.009283908511003962
trainable_parameters: 4903242
- - - - - - - - - - - - - - - - - - - - - - - - - 
  Epoch [1/122], Batch [170/176], Train Acc: 43.5179 Loss: 1.5090
  Validation Accuracy after Epoch 1: 52.4200
  Cidar10.1 Accuracy: 40.5
  Epoch [2/122], Batch [170/176], Train Acc: 60.9122 Loss: 1.2876
  Validation Accuracy after Epoch 2: 60.6000
  Cidar10.1 Accuracy: 47.85
  Epoch [3/122], Batch [170/176], Train Acc: 68.2100 Loss: 1.0749
  Validation Accuracy after Epoch 3: 69.4600
  Cidar10.1 Accuracy: 56.6
  Epoch [4/122], Batch [170/176], Train Acc: 72.9710 Loss: 1.0929
  Validation Accuracy after Epoch 4: 71.5600
  Cidar10.1 Accuracy: 58.55
  Epoch [5/122], Batch [170/176], Train Acc: 76.296

[I 2025-03-12 10:05:51,497] Trial 99 pruned. 


...

--------------------------------------------------
trial.number=121
num_epochs: 59
model_type: largeresnet
batch_size: 128
optimizer_type: AdamW
scheduler_type: OneCycleLR
beta1: 0.9029821077514644
beta2: 0.9962220109314475
lr: 0.0005211576578906421
weight_decay: 0.004782487092338308
max_lr: 0.0017541946511082217
trainable_parameters: 4903242
- - - - - - - - - - - - - - - - - - - - - - - - - 
  Epoch [1/59], Batch [350/352], Train Acc: 41.1719 Loss: 1.7003
  Validation Accuracy after Epoch 1: 47.2200
  Trial pruned due to no improvement.


[I 2025-03-12 10:24:21,127] Trial 121 pruned. 


--------------------------------------------------
trial.number=122
num_epochs: 51
model_type: base
batch_size: 512
optimizer_type: AdamW
scheduler_type: OneCycleLR
beta1: 0.8998613394428597
beta2: 0.9926581307024805
lr: 0.00011348728256968825
weight_decay: 0.003559142099931068
max_lr: 0.009872217499931642
trainable_parameters: 4903242
- - - - - - - - - - - - - - - - - - - - - - - - - 
  Epoch [1/51], Batch [80/88], Train Acc: 36.3477 Loss: 1.6744
  Validation Accuracy after Epoch 1: 42.6800
  Trial pruned due to no improvement.


[I 2025-03-12 10:24:32,291] Trial 122 pruned. 


Continued Study:
Best trial: 20
Best hyperparameters: {'num_epochs': 79, 'model_type': 'largeresnet', 'batch_size': 128, 'optimizer_type': 'AdamW', 'scheduler_type': 'OneCycleLR', 'beta1': 0.8966741749809605, 'beta2': 0.9953296889087532, 'lr': 0.0009949003229868505, 'weight_decay': 0.002220705473846143, 'max_lr': 0.005605644909963183}
Best validation accuracy: 93.92


### Single Run

In [76]:
# Realistic tranformation for better generalization
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.05),  # Mild color variations
    transforms.ToTensor(),
    transforms.Normalize(**cifar_10_mean_std),
])

model = LargeResNet0()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 79
batch_size = 128
dataset_size = 50_000 * 0.9


# Compute steps per epoch
steps_per_epoch = dataset_size // batch_size  # Integer division
if dataset_size % batch_size != 0:
    steps_per_epoch += 1  # Add 1 step if there's a remainder

optimizer_type = "AdamW"
optimizer_params = {
    "lr":0.0009949003229868505, "weight_decay": 0.002220705473846143, "betas": (0.9267890327351337, 0.9979342136957566)
}
scheduler_type = "OneCycleLR"
scheduler_params = {
    "max_lr": 0.005605644909963183, "steps_per_epoch": int(steps_per_epoch), "epochs": num_epochs, "anneal_strategy": "cos"
}
criterion_params = {"label_smoothing": 0.1}

checkpoint_fp = "studies/study_2025-03-12_02-29-51/checkpoint/trial_20_val_acc_ResNetLarge_93.9200_2025-03-12_06-08-28.pth"
checkpoint = torch.load(checkpoint_fp)
model.load_state_dict(checkpoint)

# single_run(
#     model,
#     train_transform,
#     num_epochs=num_epochs,
#     batch_size=batch_size,
#     optimizer_type=optimizer_type,
#     optimizer_params=optimizer_params,
#     scheduler_type=scheduler_type,
#     scheduler_params=scheduler_params,
#     criterion_params=criterion_params
# )

<All keys matched successfully>

### Train with pseudo labels cifar10.1 test

In [77]:
from cifar10_1_dataloader import get_dataloader_10_1
from data_loader import get_kaggle_test_dataloader
import torch.utils.data as data
from pseudo_labels import generate_pseudo_labels, PseudoDataset, CustomCIFAR10Dataset

pseudo_transform = transforms.Compose([
    transforms.RandAugment(num_ops=2, magnitude=9),  # Stronger augmentation
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(**cifar_10_mean_std)
])

train_loader, valid_loader = get_cifar10_dataloaders(
    transforms.ToTensor(), 
    batch_size=batch_size, 
    num_workers=8, 
    valid_size=0.1, 
    subset_percent=1.0
)
train_dataset = train_loader.dataset

# Convert original dataset into tensors
train_images = torch.stack([train_dataset[i][0] for i in range(len(train_dataset))])  # Stack images
train_labels = torch.tensor([train_dataset[i][1] for i in range(len(train_dataset))], dtype=torch.long)  # Convert labels to tensor
custom_train_dataset = CustomCIFAR10Dataset(train_images, train_labels, transform=train_transform)

# dataloader_10_1 = get_dataloader_10_1()
kaggle_test_data_loader = get_kaggle_test_dataloader()
pseudo_images, pseudo_labels = generate_pseudo_labels(model, kaggle_test_data_loader, threshold=0.85, device=device)
pseudo_images.shape

torch.Size([6623, 3, 32, 32])

In [78]:
# Fine-tune Model with Pseudo Labels
if pseudo_images is not None:
    pseudo_dataset = PseudoDataset(pseudo_images, pseudo_labels, transform=pseudo_transform)
    pseudo_loader = DataLoader(pseudo_dataset, batch_size=batch_size, shuffle=True)

    # Combine CIFAR-10 and pseudo-labeled data
    combined_loader = data.DataLoader(
        data.ConcatDataset([custom_train_dataset, pseudo_dataset]), batch_size=batch_size, shuffle=True)

    optimizer_params["lr"] = 5e-5
    optimizer = optimizer_map[optimizer_type](model.parameters(), **optimizer_params)
    scheduler = scheduler_map[scheduler_type](optimizer, **scheduler_params)
    criterion = nn.CrossEntropyLoss(**criterion_params)
    
    # Training
    best_val_accuracy = train_model(
        model, combined_loader, criterion, optimizer, valid_loader=valid_loader, 
        num_epochs=3, device=device, scheduler=scheduler
    )
    print(best_val_accuracy)

  Epoch [1/5], Batch [400/404], Train Acc: 90.8145 Loss: 0.6419
  Validation Accuracy after Epoch 1: 70.0600
  Cidar10.1 Accuracy: 85.3
  Epoch [2/5], Batch [400/404], Train Acc: 91.6133 Loss: 0.7215
  Validation Accuracy after Epoch 2: 74.8400
  Cidar10.1 Accuracy: 85.6
  Epoch [3/5], Batch [400/404], Train Acc: 92.0957 Loss: 0.6678
  Validation Accuracy after Epoch 3: 75.4600
  Cidar10.1 Accuracy: 85.55
  Epoch [4/5], Batch [400/404], Train Acc: 92.5078 Loss: 0.6540
  Validation Accuracy after Epoch 4: 76.7600
  Cidar10.1 Accuracy: 85.1
  Epoch [5/5], Batch [400/404], Train Acc: 92.8125 Loss: 0.6381
  Validation Accuracy after Epoch 5: 76.3000
  Cidar10.1 Accuracy: 85.15
Best Validation Accuracy: 76.7600

76.76


### Test on CIFAR10.1 data

In [79]:
from cifar10_1_dataloader import get_dataloader_10_1
dataloader_10_1 = get_dataloader_10_1(num_samples=2000)

acc, _ = evaluate_model(model, dataloader_10_1, device)
print("Acc:", acc)

Acc: 85.15


### TTA - on CIFAR10.1

In [80]:
from tta import tta_predict_batched

dataloader_10_1 = get_dataloader_10_1(num_samples=2000)

# Load test data
model.eval()

# Run TTA only on low-confidence predictions
correct, total = 0, 0
for images, labels in dataloader_10_1:
    images, labels = images.to("cuda"), labels.to("cuda")
    preds = tta_predict_batched(model, images, conf_threshold=0.8)  # Apply selective TTA
    predicted_classes = preds.argmax(dim=1)

    total += labels.size(0)
    correct += (predicted_classes == labels).sum().item()

print(f"TTA Accuracy: {100 * correct / total:.2f}%")

TTA Accuracy: 85.75%


### Test on test data

In [81]:
from trainer import evaluate_model
from data_loader import get_test_dataloader

test_loader = get_test_dataloader(use_kaggle=True)
acc, _ = evaluate_model(model, test_loader, device=device)
print("Acc:", acc)

Acc: 93.33


### Load Checkpoint

In [5]:
# model = SmallResNet0()
# model.to(device)

# # best_checkpoint_fp = "checkpoints_study_2025-03-10_19-00-59/model_trial_0_val_acc_0.8604.pth"
# best_checkpoint_fp = "studies/study_2025-03-11_01-51-41/checkpoint/trial_0_val_acc_SmallResNet_86.7000_2025-03-11_02-15-35.pth"

# if not best_checkpoint_fp:
#     checkpoint_dir = f"checkpoints_{study_name}"
#     with open(os.path.join(checkpoint_dir, "study_details.json"), "r") as f:
#         study_details = json.load(f)
#     best_checkpoint_fp = study_details[str(study.best_trial.number)]["checkpoint_path"]

# # Load the latest checkpoint
# checkpoint = torch.load(best_checkpoint_fp)
# model.load_state_dict(checkpoint)

<All keys matched successfully>

### Run model on Kaggle test data

In [31]:
from data_loader import get_kaggle_test_dataloader

In [33]:
# Generate submission file with test data
kaggle_test_loader = get_kaggle_test_dataloader()

model.eval()
predictions = []

with torch.no_grad():
    for images, in kaggle_test_loader:
        images = images.to(device)
        outputs = model(images) 
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

submission = pd.DataFrame({'ID': np.arange(len(predictions)), 'Labels': predictions})
submission.to_csv('submission.csv', index=False)
print("submission file saved.")

submission file saved.


### Run model on Kaggle test data with TTA

In [ ]:
from tta import tta_predict_batched

# dataloader_10_1 = get_dataloader_10_1()
kaggle_test_data_loader = get_kaggle_test_dataloader()

# Load test data
model.eval()

# Run TTA only on low-confidence predictions
correct, total = 0, 0
for images, labels in kaggle_test_data_loader:
    images, labels = images.to("cuda"), labels.to("cuda")
    preds = tta_predict_batched(model, images, conf_threshold=0.8)  # Apply selective TTA
    predicted_classes = preds.argmax(dim=1)

    total += labels.size(0)
    correct += (predicted_classes == labels).sum().item()

print(f"TTA Accuracy: {100 * correct / total:.2f}%")

In [6]:
from data_loader import get_kaggle_test_dataloader
from tta import tta_predict_batched

kaggle_test_loader = get_kaggle_test_dataloader()

In [7]:
model.eval()
predictions = []

with torch.no_grad():
    for images, in kaggle_test_loader:
        images = images.to(device)
        preds = tta_predict_batched(model, images, conf_threshold=0.8)  # Apply selective TTA
        predicted = preds.argmax(dim=1)
        predictions.extend(predicted.cpu().numpy())

submission = pd.DataFrame({'ID': np.arange(len(predictions)), 'Labels': predictions})
submission.to_csv('submission.csv', index=False)
print("submission file saved.")

submission file saved.


In [35]:
# import kaggle
# kaggle.api.competition_submit(
#     file_name="submission.csv",
#     message="0.9365",
#     competition="deep-learning-spring-2025-project-1"
# )

100%|██████████| 67.3k/67.3k [00:00<00:00, 330kB/s]


Successfully submitted to Deep Learning Spring 2025: CIFAR 10 classification